![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to analyze car rental customer satisfaction and detect business area.

**Note:** Please note that for the watsonx challenge, please run these notebooks locally on your laptop/desktop and do not run it in IBM Cloud.  The instructions for running the notebook locally are provided in the readme.md file present in the zip file.

This notebook contains the steps and code to demonstrate support of text sentiment analysis in Watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

<a id="setup"></a>
## Set up the environment

### Install and import the dependecies

**Note:** For Windows environments, please remove `| tail -n 1` commands in the cell below.

In [16]:
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install ibm-watson-machine-learning==1.0.312 | tail -n 1


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


**Note:** Please restart the notebook kernel to pick up proper version of packages installed above.

In [1]:
import os, getpass
from pandas import read_csv

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [2]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter): ········


### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [17]:
try:
    project_id = "b3912d60-3e2c-49f1-975c-3e2f12d71394" #os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [18]:
filename_test = 'test.csv'
filename_train = 'train.csv'

test_data = read_csv(filename_test)
train_data = read_csv(filename_train)

In [19]:
train_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2944,Female,M,2,41.92,Active,No,Customer service was friendly and helpful.,1,Service: Knowledge,NaN
1,1119,Female,M,2,33.60,Active,Yes,Customer service was good at MSP airport and t...,1,Service: Knowledge,NaN
2,0,Male,M,0,51.00,Inactive,Yes,I do not understand why I have to pay additio...,0,Product: Pricing and Billing,Premium features
3,1085,Female,S,2,42.00,Inactive,No,Based on the customer service personnel I enco...,0,Service: Attitude,On-demand pickup location
4,0,Female,M,2,44.10,Active,No,Provide more convenient car pickup from the ai...,0,Service: Orders/Contracts,On-demand pickup location


In [20]:
test_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2771,Female,M,2,49.99,Inactive,No,"last time I rented a car was at Manchester, NH...",0,Product: Functioning,On-demand pickup location
1,1133,Male,S,1,56.05,Inactive,No,Please lower the prices.,0,Product: Pricing and Billing,Free Upgrade
2,900,Female,M,1,64.64,Active,No,Excellent response dealing with child seat.,1,Service: Accessibility,NaN
3,3795,Male,M,0,46.51,Inactive,No,"all went quite smoothly... it was Enterprise, ...",1,Service: Accessibility,NaN
4,3541,Male,S,1,17.01,Inactive,Yes,"Slow, long lineup",0,Product: Functioning,On-demand pickup location


In [21]:
len(test_data)

50

<a id="models"></a>
## Foundation Models on Watsonx

You need to specify `model_id` that will be used for inferencing.

**Action**: Use `FLAN_UL2` model.

In [22]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [23]:
model_id = ModelTypes.FLAN_UL2

<a id="predict"></a>
## Evaluate the model, prompt and parameters

### **1. Customer satisfaction**

Define instructions for the model to recognize if customer was satisfied or unsatisfied.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [24]:
satisfaction_instruction = """
   Was customer satisfied?\n

   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be\n
   satisfaction: 0\n\n
"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [25]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.MAX_NEW_TOKENS: 10
}

Initialize the `Model` class.

In [26]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

Analyze the customer satisfaction for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [27]:
results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    input = " ".join([satisfaction_instruction, input_text])
#     print(input)
    results.append(model.generate_text(prompt=input))


   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

   satisfaction: 0


 last time I rented a car was at Manchester, NH airport and they do not have office there anymore

   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

   satisfaction: 0


 Please lower the prices.

   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and t


   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

   satisfaction: 0


 fast service

   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

   satisfaction: 0


 They had problem to find reservation number, so whole process took too long time.

   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longe


   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

   satisfaction: 0


 Customer Service was quick and friendly.  I received my car with little hassles.

   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be

   satisfaction: 0


 The person who assisted me was not very friendly and didn't like to give the details I was asking for (gas refill packages, insurance, etc.).

   Was customer satisfied?


   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apolo

In [28]:
print(results)

['no', 'dissatisfied', 'satisfaction: 5', 'satisfaction: 4', 'no', 'satisfaction: 0', 'satisfaction: 3', 'no', 'satisfaction: 5', 'satisfaction: 5', 'satisfaction: 3', 'no', 'dissatisfaction: 1', 'satisfaction: 4', 'satisfaction: 3', 'satisfaction: 0', 'no', 'no', 'dissatisfied', 'satisfaction: 5', 'no', 'no', 'satisfaction: 0', 'satisfaction: 5', 'satisfaction: 3', 'satisfaction: 4', 'satisfaction: 5', 'satisfaction: 0', 'satisfaction: 4', 'satisfaction: 4', 'no', 'satisfaction: 3', 'satisfaction: 5', 'satisfaction: 1', 'satisfaction: 0', 'dissatisfaction', 'satisfaction: 3', 'satisfaction: 3', 'satisfaction: 1', 'satisfaction: 4', 'satisfaction: 5', 'no', 'satisfaction: 3', 'satisfaction: 0', 'no', 'no', 'satisfaction: 1', 'satisfaction: 4', 'satisfaction: 5', 'satisfaction: 3']


### Calculate the F1 micro score

In [29]:
from sklearn.metrics import f1_score

print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

f1_micro_score 0.0


In [33]:
satisfaction_instruction = """
   Was customer satisfied? if satisfaction larger than 3 return 1 otherwise 0.\n

   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be\n
   satisfaction: 0\n\n
"""
results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    input = " ".join([satisfaction_instruction, input_text])
    results.append(model.generate_text(prompt=input))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']
f1_micro_score 0.92


In [34]:
satisfaction_instruction = """
   Was customer satisfied? if satisfaction larger than 2 return 1 otherwise 0.\n

   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be\n
   satisfaction: 0\n\n
"""
results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    input = " ".join([satisfaction_instruction, input_text])
    results.append(model.generate_text(prompt=input))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']
f1_micro_score 0.92


In [35]:
satisfaction_instruction = """
   Was customer satisfied? if satisfaction larger than 3 return 1 otherwise 0.\n
"""
results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    input = " ".join([satisfaction_instruction, input_text])
    results.append(model.generate_text(prompt=input))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']
f1_micro_score 0.94


In [38]:
satisfaction_instruction = """
   Was customer satisfied? if satisfaction larger than 0 return 1 otherwise 0.\n
"""
results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    input = " ".join([satisfaction_instruction, input_text])
    results.append(model.generate_text(prompt=input))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']
f1_micro_score 0.92


In [102]:
train_comments = list(train_data.Customer_Service)
train_satisfaction = list(train_data.Satisfaction.astype(str))

train_array_sent = []
for idx, text in enumerate(train_comments):
    train_array_sent.append("comment: " + text + "\nsatisfaction: " + train_satisfaction[idx] )
    
train_str = "\n\n".join(train_array_sent)
print(train_str)

comment: Customer service was friendly and helpful.
satisfaction: 1

comment: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
satisfaction: 1

comment: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
satisfaction: 0

comment: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
satisfaction: 0

comment: Provide more convenient car pickup from the airport parking.
satisfaction: 0

comment: all went quite without hassle, they picked me up to get the car it was great.
satisfaction: 1

comment: VERY slow service!
satisfaction: 0

comment: Its nice when you get in your car and it has already been started up with the A/C or heating go

In [60]:
train_str_10 = "\n\n".join(train_array[:10])
print(train_str_10)

comment: Customer service was friendly and helpful.
satisfaction: 1

comment: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
satisfaction: 1

comment: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
satisfaction: 0

comment: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
satisfaction: 0

comment: Provide more convenient car pickup from the airport parking.
satisfaction: 0

comment: all went quite without hassle, they picked me up to get the car it was great.
satisfaction: 1

comment: VERY slow service!
satisfaction: 0

comment: Its nice when you get in your car and it has already been started up with the A/C or heating go

In [30]:
satisfaction_instruction = "what is the customer satisfaction sentiment? show 0 for negative and 1 for positive.\n"

results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    results.append(model.generate_text(prompt=" ".join([satisfaction_instruction, input_text])))

print(results)

['0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']


In [31]:
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

f1_micro_score 0.94


In [65]:
satisfaction_instruction = "what is the customer satisfaction sentiment? show 0 for negative and 1 for positive.\n"

train_str_10 = "\n\n".join(train_array[:10])

results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    prompt="\n".join([satisfaction_instruction, train_str_10, input_text])
#     print(prompt)
    results.append(model.generate_text(prompt=prompt))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '0', '1', '1', '1', '1', 'satisfaction: 1', 'satisfaction: 1', '0', '1', '1', '0', 'satisfaction: 1', '0', '0', 'satisfaction: 1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', 'satisfaction: 1', '1', '1', '0', '0', 'satisfaction: 1', '1', '0', '1', '1', '0', 'satisfaction: 1', '0', '0', 'satisfaction: 1', '1', '1', '1', '1']
f1_micro_score 0.78


In [67]:
print([r[-1:] for r in results])

['0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']


In [68]:
print('f1_micro_score', f1_score(satisfaction, [r[-1:] for r in results], average='micro'))

f1_micro_score 0.94


In [69]:
satisfaction_instruction = "what is the customer satisfaction sentiment? show 0 for negative and 1 for positive.\n"

results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    prompt="\n".join([satisfaction_instruction, train_str, input_text])
#     print(prompt)
    results.append(model.generate_text(prompt=prompt))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']
f1_micro_score 0.92


In [50]:
satisfaction_instruction = "what is commment sentiment? show 0 for negative and 1 for positive.\n"

results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    prompt="\n".join([satisfaction_instruction, "comment: "+input_text, "sentiment: "])
    #prompt="\n".join([satisfaction_instruction, input_text])
    #print(prompt)
    results.append(model.generate_text(prompt=prompt))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']
f1_micro_score 0.96


In [51]:
satisfaction_instruction = "what is commment sentiment? show 0 for negative and 1 for positive.\n"

train_comments = list(train_data.Customer_Service)
train_satisfaction = list(train_data.Satisfaction.astype(str))

train_array_sent = []
for idx, text in enumerate(train_comments):
    train_array_sent.append("comment: " + text + "\nsentiment: " + train_satisfaction[idx] )
    
train_array_sent_str = "\n\n".join(train_array_sent)
# print(train_array_sent_str)

results = []
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    prompt="\n".join([satisfaction_instruction, "\n\n".join(train_array_sent), "\ncomment: "+input_text, "sentiment: "])
#     print(prompt)
    results.append(model.generate_text(prompt=prompt))

print(results)
print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

['0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1']
f1_micro_score 0.94


### **2. Business area**

Define instructions for the model to detect business area in which the customer review is targeted.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [52]:
business_area_instruction = """
   Find the business area of the customer e-mail. Choose business area from the following list: 'Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts'.\n

   comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be\n
   business area: 'Product: Availability/Variety/Size'\n\n
"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [53]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.MAX_NEW_TOKENS: 15
}

Initialize the `Model` class.

In [54]:
from ibm_watson_machine_learning.foundation_models import Model

model2 = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

Analyze the recommended actions for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [55]:
results = []
comments = list(test_data.Customer_Service)
area = list(test_data.Business_Area)

for input_text in comments:
    results.append(model2.generate_text(prompt=" ".join([business_area_instruction, input_text])))

In [56]:
print(results)

['Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Availability/Variety/Size', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Product: Availability/Variety/Size', 'Product: Availability/Variety/Size', 'Product: Functioning', 'Service: Attitude', 'Product: Availability/Variety/Size', 'Service: Attitude', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Pricing and Billing', 'Product: Availability/Variety/Size', 'Service: Knowledge', 'Service: Attitude', 'Service: Attitude', 'Product: Functioning', 'Product: Availability/Variety/Size', 'Service: Attitude', 'Product: Availability/Variety/Size', 'Service: Attitude', 'Service: Attitude', 'Product: Availability/Variety/Size', 'Service: Attitude', 'Service: Attitude', 'Product: Availability/Variety/Size', 'Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Prod

### Calculate the F1 micro score

In [57]:
from sklearn.metrics import f1_score

print('f1_micro_score', f1_score(area, results, average='micro'))

f1_micro_score 0.06


In [58]:
results = []
comments = list(test_data.Customer_Service)[:20]
area = list(test_data.Business_Area)[:20]

for input_text in comments:
    results.append(model2.generate_text(prompt=" ".join([business_area_instruction, input_text])))

print(results)
print('f1_micro_score', f1_score(area, results, average='micro'))

['Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Availability/Variety/Size', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Product: Availability/Variety/Size', 'Product: Availability/Variety/Size', 'Product: Functioning', 'Service: Attitude', 'Product: Availability/Variety/Size', 'Service: Attitude', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Pricing and Billing', 'Product: Availability/Variety/Size']
f1_micro_score 0.15


In [59]:
business_area_instruction = """
   Find the business area of the customer e-mail. Choose business area from the following list: 'Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts'.\n
"""

results = []
comments = list(test_data.Customer_Service)[:20]
area = list(test_data.Business_Area)[:20]

for input_text in comments:
    results.append(model2.generate_text(prompt=" ".join([business_area_instruction, input_text])))

print(results)
print('f1_micro_score', f1_score(area, results, average='micro'))

['Service: Accessibility', 'Product: Pricing and Billing', 'Service: Attitude', 'Service: Accessibility', 'Service: Accessibility', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Service: Accessibility', 'Product: Information', 'Product: Functioning', 'Service: Attitude', 'Service: Attitude', 'Service: Attitude', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Pricing and Billing', 'Service: Attitude']
f1_micro_score 0.20000000000000004


In [88]:
print(str(set(test_data.Business_Area)))

{'Service: Attitude', 'Service: Accessibility', 'Service: Orders/Contracts', 'Product: Information', 'Product: Pricing and Billing', 'Product: Availability/Variety/Size', 'Service: Knowledge', 'Product: Functioning'}


In [90]:
business_area_instruction = """
   Find the business area of the customer e-mail. Choose business area from the following list: 'Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts'.
"""
   
train_examples = ""

results = []
for input_text in list(test_data.Customer_Service):
    results.append(model2.generate_text(prompt=" ".join([business_area_instruction, train_examples, input_text])))
   
# print(results)

print('f1_micro_score', f1_score(list(test_data.Business_Area), results, average='micro'))

f1_micro_score 0.18


In [61]:
train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("email: " + text + "\nbusiness area: '" + train_ba[idx] + "'")
    
train_str = "\n\n".join(train_array)
# print(train_str)

In [63]:
business_area_instruction = """
   Find the business area of the customer e-mail. Choose business area from the following list: 'Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts'.
"""

size = 20

train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\nemail: "+input_text+"\nbusiness area: "])
#     print(text)
    results.append(model2.generate_text(prompt=text))
   
print(list(test_data[:size].Business_Area))
print(results)

print('f1_micro_score', f1_score(list(test_data[:size].Business_Area), results, average='micro'))

['Product: Functioning', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Accessibility', 'Product: Functioning', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Service: Orders/Contracts']
['Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing',

In [67]:
business_area_instruction = """
   Find the business area of the customer e-mail. Choose business area from the following list: 'Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts'.
"""

train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("email: " + text + "\nba: '" + train_ba[idx] + "'")
    
train_str = "\n\n".join(train_array)
# print(train_str)


size = 20

train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\nemail: "+input_text+"\ba: "])
#     print(text, len(text))
    results.append(model2.generate_text(prompt=text))
   
print(list(test_data[:size].Business_Area))
print(results)

print('f1_micro_score', f1_score(list(test_data[:size].Business_Area), results, average='micro'))

['Product: Functioning', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Accessibility', 'Product: Functioning', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Service: Orders/Contracts']
['Product: Pricing and Billing', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Pricing and Bil

In [95]:
business_area_instruction = """
Firstly, classify each case as 'Product' or 'Service'.
'Product' is usually related to cases in which the client talks about the car. Otherwise it is 'Service'.
Next, attribute business areas for each customer message about the car rent service, by choosing the most proper business area. 
For 'Product' choice from ['Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing'].\n
Otherwise, for 'Service' chose from ['Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts']
Consider the definitions below as guide to perform the classification:
'Service: Accessibility': cases in which the user got a good and accesible price and their car worked seamlessly. 
'Service: Knowledge': cases in which the user is informing its opinion about the attention received by employees during the rental.\n
"""

train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("email: " + text + "\nba: '" + train_ba[idx] + "'")
    
train_str = "\n\n".join(train_array)
# print(train_str)


size = 20

train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\nemail: "+input_text+"\nba: "])
#     print(text, len(text))
    results.append(model2.generate_text(prompt=text))
   
print(list(test_data[:size].Business_Area))
print(results)

print('f1_micro_score', f1_score(list(test_data[:size].Business_Area), results, average='micro'))

['Product: Functioning', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Accessibility', 'Product: Functioning', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Service: Orders/Contracts']
['Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Pricing and Billing', 'Product: Pricing and Billi

In [94]:
business_area_instruction = """
   Firstly, classify each case as 'Product' or 'Service'.
   'Product' is usually related to cases in which the client talks about the car.
   Otherwise it is 'Service'.
   Next, attribute business areas for each customer message about the car rent service, by choosing the most proper business area. 
   For 'Product' choice from ['Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing'].\n
   Otherwise, for 'Service' chose from ['Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts']
   Consider the definitions below as guide to perform the classification:
   'Service: Accessibility': cases in which the user got a good and accesible price and their car worked seamlessly. 
   'Service: Knowledge': cases in which the user is informing its opinion about the attention received by employees during the rental.\n
"""

train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("comment: " + text + "\n" + train_ba[idx] + "")
    
train_str = "\n\n".join(train_array)
# print(train_str)

size = 20

train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\ncomment: "+input_text+"\n"])
#     print(text, len(text))
    results.append(model2.generate_text(prompt=text))
   
print(list(test_data[:size].Business_Area))
print(results)

print('f1_micro_score', f1_score(list(test_data[:size].Business_Area), results, average='micro'))

['Product: Functioning', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Accessibility', 'Product: Functioning', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Service: Orders/Contracts']
['Product: Pricing and Billing', 'Product: Pricing and Billing', 'Service: Knowledge', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Functioning', 'Service: Knowledge', 'Product: Pricing and Billing', 'Service: Knowledge', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Product: Pricing and Billing', 'Service: Knowledge', 'Product: Pricing and Billing', 'Service: Knowledge', 'Product: Pricing and Billing', 'Product: Pricing and B

In [87]:
business_area_instruction = """
   Firstly, classify each case as 'Product' or 'Service'.
   'Product' is usually related to cases in which the client talks about the car.
   Otherwise it is 'Service'.
   Next, attribute business areas for each customer message about the car rent service, by choosing the most proper business area. 
   For 'Product' choice from ['Product: Availability/Variety/Size', 'Product: Functioning', 'Product: Information', 'Product: Pricing and Billing'].\n
   Otherwise, for 'Service' chose from ['Service: Accessibility', 'Service: Attitude', 'Service: Knowledge', 'Service: Orders/Contracts']
   Consider the definitions below as guide to perform the classification:
   'Service: Accessibility': cases in which the user got a good and accesible price and their car worked seamlessly. 
   'Service: Knowledge': cases in which the user is informing its opinion about the attention received by employees during the rental.\n
   'Product: Availability/Variety/Size': cases related to the availability  and offering a car.
   'Product: Functioning': cases that considers experience of the customer. Involves staff, agent service, treatment, personel. It is related . key words: helpful, experience,  custumer service, order, lines, wait time, free upgrade
   'Product: Information': cases related to directions to a location.
   'Product: Pricing and Billing': cases related to price, payment and cost of the product. key words: expensive, price
"""

train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("comment: " + text + "\n" + train_ba[idx] + "")
    
train_str = "\n\n".join(train_array)
# print(train_str)

size = 20

train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\ncomment: "+input_text+"\n"])
#     print(text, len(text))
    results.append(model2.generate_text(prompt=text))
   
print(list(test_data[:size].Business_Area))
print(results)

print('f1_micro_score', f1_score(list(test_data[:size].Business_Area), results, average='micro'))

['Product: Functioning', 'Product: Pricing and Billing', 'Service: Accessibility', 'Service: Accessibility', 'Product: Functioning', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Service: Knowledge', 'Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Service: Orders/Contracts']
['Product: Availability/Variety/Size', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Product: Functioning', 'Service: Knowledge', 'Product: Functioning', 'Service: Knowledge', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Functioning', 'Product: Pricing and Billing', 'Product: Pricing and Billing', 'Product: Prici

In [96]:
categories = []
for t in set(train_data.Business_Area):
    categories.append(t.split(":")[1].strip())

business_area_instruction = "Classify an email in the following categories: " + ", ".join(categories)

train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("email: " + text + "\ntype: " + train_ba[idx].split(":")[1].strip() + "")
    
train_str = "\n\n".join(train_array)
# print(train_str)

size = 20

train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\nemail: "+input_text+"\ntype: "])
    print(text, len(text))
    results.append(model2.generate_text(prompt=text))
   
# print(list(test_data[:size].Business_Area))

expected = []
for t in list(test_data[:size].Business_Area):
    expected.append(t.split(":")[1].strip())

print(expected)
print(results)

print('f1_micro_score', f1_score(expected, results, average='micro'))

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

...

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

Classify an email in the following categories: Information, Pricing and Billing, Availability/Variety/Size, Knowledge, Orders/Contracts, Functioning, Accessibility, Attitude
email: Customer service was friendly and helpful.
type: Knowledge

email: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
type: Knowledge

email: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
type: Pricing and Billing

email: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
type: Attitude

email: Provide more convenient car pickup from the airport parking.
type: Orders/Contracts

email: all went quite without hassle, they picked me up to get the car it

['Functioning', 'Pricing and Billing', 'Accessibility', 'Accessibility', 'Functioning', 'Functioning', 'Knowledge', 'Knowledge', 'Functioning', 'Knowledge', 'Knowledge', 'Knowledge', 'Functioning', 'Knowledge', 'Knowledge', 'Knowledge', 'Knowledge', 'Availability/Variety/Size', 'Pricing and Billing', 'Orders/Contracts']
['Information', 'Pricing and Billing', 'Knowledge', 'Functioning', 'Functioning', 'Attitude', 'Attitude', 'Attitude', 'Knowledge', 'Knowledge', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Pricing and Billing', 'Pricing and Billing', 'Functioning']
f1_micro_score 0.25


In [106]:
categories = []
for t in set(train_data.Business_Area):
    categories.append(t.split(":")[0].strip())

business_area_instruction = "Classify an email text in the following types: " + ", ".join(categories)

train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("email: " + text + "\ntype: " + train_ba[idx].split(":")[0].strip() + "")
    
train_str = "\n\n".join(train_array)
# print(train_str)

size = 50

train_ba_text = ""#"\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\nemail: "+input_text+"\ntype: "])
    #print(text, len(text))
    results.append(model2.generate_text(prompt=text))
   
# print(list(test_data[:size].Business_Area))

expected = []
for t in list(test_data[:size].Business_Area):
    expected.append(t.split(":")[0].strip())

print(expected)
print(results)

print('f1_micro_score', f1_score(expected, results, average='micro'))

['Product', 'Product', 'Service', 'Service', 'Product', 'Product', 'Service', 'Service', 'Product', 'Service', 'Service', 'Service', 'Product', 'Service', 'Service', 'Service', 'Service', 'Product', 'Product', 'Service', 'Product', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Product', 'Service', 'Service', 'Product', 'Product', 'Service', 'Service', 'Service', 'Service', 'Service', 'Product', 'Product', 'Service', 'Service', 'Product', 'Service', 'Service', 'Service', 'Service', 'Service', 'Product', 'Product', 'Service']
['Service', 'Product', 'Service', 'Service', 'Service', 'Service', 'Service', 'Product', 'Service', 'Service', 'Service', 'Product', 'Product', 'Service', 'Product', 'Service', 'Product', 'Service', 'Product', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Service', 'Product', 'Service', 'Service', 'Product', 'Service', 'Service', 'Service

In [105]:
categories = []
for t in set(train_data.Business_Area):
    categories.append(t.split(":")[1].strip())

business_area_instruction = f"""
Classify an email text in the following types: {", ".join(categories)}. Where: 
Accessibility are cases in which the user got a good and accesible price and their car worked seamlessly; 
Knowledge are cases in which the user is informing its opinion about the attention received by employees during the rental;
Availability/Variety/Size are cases related to the availability  and offering a car;
Functioning are cases that considers experience of the customer, involves staff, agent service, treatment, personel, it is related with helpful, experience, custumer service, order, lines, wait time, free upgrade;
Information are cases related to directions to a location;
Pricing and Billing are cases related to price, payment and cost of the product for example expensive cost or high price
"""

train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("email: " + text + "\ntype: " + train_ba[idx].split(":")[1].strip() + "")
    
train_str = "\n\n".join(train_array)
# print(train_str)

size = 20

train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, train_ba_text, "\nemail: "+input_text+"\ntype: "])
    #print(text, len(text))
    results.append(model2.generate_text(prompt=text))
   
# print(list(test_data[:size].Business_Area))

expected = []
for t in list(test_data[:size].Business_Area):
    expected.append(t.split(":")[1].strip())

print(expected)
print(results)

print('f1_micro_score', f1_score(expected, results, average='micro'))

['Functioning', 'Pricing and Billing', 'Accessibility', 'Accessibility', 'Functioning', 'Functioning', 'Knowledge', 'Knowledge', 'Functioning', 'Knowledge', 'Knowledge', 'Knowledge', 'Functioning', 'Knowledge', 'Knowledge', 'Knowledge', 'Knowledge', 'Availability/Variety/Size', 'Pricing and Billing', 'Orders/Contracts']
['Information', 'Pricing and Billing', 'Knowledge', 'Functioning', 'Functioning', 'Knowledge', 'Knowledge', 'Attitude', 'Knowledge', 'Knowledge', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Functioning', 'Pricing and Billing', 'Pricing and Billing', 'Functioning']
f1_micro_score 0.3


In [135]:
categories = []
for t in set(train_data.Business_Area):
    categories.append(t) #.split(":")[1].strip())

    # car rental
train_email = list(train_data.Customer_Service)
train_ba = list(train_data.Business_Area)

train_array = []
for idx, text in enumerate(train_email):
    train_array.append("Comment: " + text + "\nName of the category: " + train_ba[idx].strip() + "")
    
train_str = "\n\n".join(train_array)
# print(train_str)
    
business_area_instruction = f"""
Act as a business analyst of a car rental company. Your company wants to classify every client comment about its last rental in one of the categories described below. 

Name: Product: Availability/Variety/Size
Description: cases related to the availability and offering a car and agency.

Name: Product: Functioning
Description: cases that considers experience of the customer with the rental car. Involves staff and agent service and the way the treatment goes. It is related to experience, wait lines or time, free upgrade and insurrance

Name: Product: Information
Description: cases related to directions on how to get to pick up or drop off location, including maps and indication

Name: Product: Pricing and Billing
Description: cases related to price, payment and cost of the product, for example expensive cost, extra charge or high price.

Name: Service: Attitude
Description: cases in which the hard work of the staff is enphasised , specially related to taking care of the customer

Name: Service: Accessibility
Description: cases in which teh client describes the experience on his arrival

Name: Service: Knowledge
Description: cases in which the user is informing its opinion about the attention received by employees (people) during the rental and the staff apperance and agents help. It als describe how well the staff was in terms of behavior, such as nice or fine, and knowledgeable. it is the overall customer services

Name: Service: Orders/Contracts
Description: is describes incosistence on what is the service expected versus the services received. Includes complains on  pickup airport and drive experience

Use the following examples as reference:
{train_str}

The following paragraph is a client comment. The comment must be classified in one of the categories described above. Read the following comment and determine which category it is about. Make sure to use exactly one of the names of the categories listed above. 

"""

size = 50

# train_ba_text = "\n\n".join(train_array)

results = []
for input_text in list(test_data[:size].Customer_Service):
    text = "\n".join([business_area_instruction, input_text, "Name of the category: "])
    #print(text, len(text))
    results.append(model2.generate_text(prompt=text))

expected = []
for t in list(test_data[:size].Business_Area):
    expected.append(t.strip()) #.split(":")[1].strip())

# print(expected)
# print(results)

print('f1_micro_score overall:', f1_score(expected, results, average='micro'))

import pandas as pd

results_df = pd.DataFrame({'expected': expected, 'results': results })
for i, g in results_df.groupby('expected'):
    print(i, len(g))
    print('f1_micro_score', f1_score(list(g.expected), list(g.results), average='micro'), "\n")

Comment: Customer service was friendly and helpful.
Name of the category: Service: Knowledge

Comment: Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.
Name of the category: Service: Knowledge

Comment: I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
Name of the category: Product: Pricing and Billing

Comment: Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
Name of the category: Service: Attitude

Comment: Provide more convenient car pickup from the airport parking.
Name of the category: Service: Orders/Contracts

Comment: all went quite without hassle, they picked me up to get the car it was great.
Name of the category: Ser

f1_micro_score overall: 0.3
Product: Availability/Variety/Size 1
f1_micro_score 0.0 

Product: Functioning 12
f1_micro_score 0.25 

Product: Information 1
f1_micro_score 1.0 

Product: Pricing and Billing 3
f1_micro_score 1.0 

Service: Accessibility 7
f1_micro_score 0.0 

Service: Attitude 1
f1_micro_score 0.0 

Service: Knowledge 20
f1_micro_score 0.4000000000000001 

Service: Orders/Contracts 5
f1_micro_score 0.0 



In [119]:
# print(expected)
# print(results)

import pandas as pd

results_df = pd.DataFrame(
    {'expected': expected,
     'results': results
    })


results_df

,expected,results
0,Product: Functioning,Product: Availability/Variety/Size
1,Product: Pricing and Billing,Product: Pricing and Billing
2,Service: Accessibility,Service: Knowledge
3,Service: Accessibility,Product: Functioning
4,Product: Functioning,Product: Functioning
5,Product: Functioning,Product: Functioning
6,Service: Knowledge,Service: Attitude
7,Service: Knowledge,Service: Knowledge
8,Product: Functioning,Service: Knowledge
9,Service: Knowledge,Service: Knowledge


In [134]:
results_df.groupby('expected')

for i, g in results_df.groupby('expected'):
    print(i, len(g))
#     print(list(g.expected))
    print('f1_micro_score', f1_score(list(g.expected), list(g.results), average='micro'), "\n")

Product: Availability/Variety/Size 1
f1_micro_score 0.0 

Product: Functioning 12
f1_micro_score 0.25 

Product: Information 1
f1_micro_score 1.0 

Product: Pricing and Billing 3
f1_micro_score 1.0 

Service: Accessibility 7
f1_micro_score 0.0 

Service: Attitude 1
f1_micro_score 0.0 

Service: Knowledge 20
f1_micro_score 0.35 

Service: Orders/Contracts 5
f1_micro_score 0.0 



---

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.